# 4. PiNN as a Neural Network Potential

In this section we'll perform some compuations with pre-trained neural network potential

In [ ]:
import tensorflow as tf
from ase import units
from pinn.models import potential_model
from pinn.calculator import PiNN_calc
from helpers import *
tf.logging.set_verbosity('ERROR')

A potential is just like the model you trained.
And the good news is you can also get all the gradients from the potential like the force, stress, hessian etc
(remember `tf.gradients()`?).

Though, to sample the potential energy surface we need more than the stable organic molecules in QM9 dataset.
Here the model is trained on the 
[ANI-1](https://figshare.com/articles/ANI-1_data_set_20M_DFT_energies_for_non-equilibrium_small_molecules/5287732) 
(20M DFT energies for non-equilibrium small molecules) dataset.
It takes longer to train and the dataset is much larger (the dataset is 4.8GB). 
We'll provide the trained model here (you can still see the training log via tensorboard).

In [ ]:
params = {'model_dir': './model_dir/PiNN_ANI',
          'network': 'pinn_network',
          'netparam': {'atom_types':[1, 6, 7, 8], 'depth': 6},
          'train': {}}

model = potential_model(params)
calc = PiNN_calc(model, unit=units.kcal/units.mol)

the model can be exported to an ASE calculator easily, and you can readily calculate the energy and forces of atoms.

In [ ]:
from ase.collections import s22
from ase import units

atoms = s22['Phenol_dimer']
atoms.set_calculator(calc)
atoms.get_potential_energy()
atoms.get_forces()

## Run some molecular dynamics

ASE also support some basic molecular dynamics implementations:
https://wiki.fysik.dtu.dk/ase/tutorials/md/md.html

For example:

In [ ]:
from ase.md.verlet import VelocityVerlet
dyn = VelocityVerlet(atoms, dt=1 * units.fs, trajectory='md.traj')
dyn.run(1000)

In [ ]:
from ase.io import read
traj = read('md.traj', index=':')
show_traj(traj)

Or perhaps simulate a large box of water?
This could take forever on a modern supercomputer, but will only take several minites with the neural network potential.

In [ ]:
atoms = read('waterbox.xyz')
atoms.set_calculator(calc)
dyn = VelocityVerlet(atoms, dt=1 * units.fs, trajectory='waterbox.traj')
dyn.run(1000)

In [ ]:
traj = read('waterbox.traj', index=':')
for atom in traj:
    atom.positions = atom.get_positions(wrap=True)
show_traj(traj)